# Structure

In [ ]:
import json
import sys

from konfoo import *

## YAML Support

In [ ]:
import oyaml as yaml

In [3]:
def to_yaml(container, *attributes, **options):
    flow_style = options.pop('flow_style', False)
    return yaml.safe_dump(container.view_fields(*attributes, **options), 
                          stream=sys.stdout, 
                          default_flow_style=flow_style)

## Define a Structure

In [4]:
class Identifier(Structure):

    def __init__(self):
        super().__init__()        # <- NEVER forget to call it first!
        self.version = Decimal(8) # 1st field
        self.id = Decimal(8)      # 2nd field
        self.length = Decimal(8)  # 3rd field
        self.module = Char()      # 4th field
        self.index_fields()       # <- Indexes all fields (optional)

In [5]:
structure = Identifier()

In [6]:
structure.to_list()

[('Identifier.version', 0),
 ('Identifier.id', 0),
 ('Identifier.length', 0),
 ('Identifier.module', '\x00')]

In [7]:
structure.to_csv()

[{'id': 'Identifier.version', 'value': 0},
 {'id': 'Identifier.id', 'value': 0},
 {'id': 'Identifier.length', 'value': 0},
 {'id': 'Identifier.module', 'value': '\x00'}]

In [8]:
structure.to_json()

'{"version": 0, "id": 0, "length": 0, "module": "\\u0000"}'

In [9]:
to_yaml(structure, flow_style=False)

version: 0
id: 0
length: 0
module: "\0"


## Align Fields in a Structure

In [10]:
class Identifier(Structure):

    def __init__(self):
        super().__init__()
        self.version = Decimal(8, align_to=4)
        self.id = Decimal(8, align_to=4)
        self.length = Decimal(8, align_to=4)
        self.module = Char(align_to=4)
        self.index_fields()

In [11]:
structure = Identifier()

In [12]:
structure.to_list('alignment')

[('Identifier.version', Alignment(byte_size=4, bit_offset=0)),
 ('Identifier.id', Alignment(byte_size=4, bit_offset=8)),
 ('Identifier.length', Alignment(byte_size=4, bit_offset=16)),
 ('Identifier.module', Alignment(byte_size=4, bit_offset=24))]

In [13]:
structure.to_csv('alignment.byte_size', 'alignment.bit_offset', fieldnames=('id', 'size', 'offset'))

[{'id': 'Identifier.version', 'size': 4, 'offset': 0},
 {'id': 'Identifier.id', 'size': 4, 'offset': 8},
 {'id': 'Identifier.length', 'size': 4, 'offset': 16},
 {'id': 'Identifier.module', 'size': 4, 'offset': 24}]

In [14]:
structure.to_json('alignment')

'{"version": [4, 0], "id": [4, 8], "length": [4, 16], "module": [4, 24]}'

## Inherit from a Structure

In [15]:
class HeaderV1(Structure):
    
    def __init__(self):
        super().__init__()
        self.type = Identifier()
        self.index_fields()

In [16]:
header = HeaderV1()

In [17]:
header.to_list()

[('HeaderV1.type.version', 0),
 ('HeaderV1.type.id', 0),
 ('HeaderV1.type.length', 0),
 ('HeaderV1.type.module', '\x00')]

In [18]:
header.to_csv()

[{'id': 'HeaderV1.type.version', 'value': 0},
 {'id': 'HeaderV1.type.id', 'value': 0},
 {'id': 'HeaderV1.type.length', 'value': 0},
 {'id': 'HeaderV1.type.module', 'value': '\x00'}]

In [19]:
header.to_json()

'{"type": {"version": 0, "id": 0, "length": 0, "module": "\\u0000"}}'

In [20]:
to_yaml(header, flow_style=False)

type:
  version: 0
  id: 0
  length: 0
  module: "\0"


In [21]:
class HeaderV2(HeaderV1):

    def __init__(self):
        super().__init__()
        self.size = Decimal32()
        self.index_fields()

In [22]:
header = HeaderV2()

In [23]:
header.to_list()

[('HeaderV2.type.version', 0),
 ('HeaderV2.type.id', 0),
 ('HeaderV2.type.length', 0),
 ('HeaderV2.type.module', '\x00'),
 ('HeaderV2.size', 0)]

In [24]:
header.to_csv()

[{'id': 'HeaderV2.type.version', 'value': 0},
 {'id': 'HeaderV2.type.id', 'value': 0},
 {'id': 'HeaderV2.type.length', 'value': 0},
 {'id': 'HeaderV2.type.module', 'value': '\x00'},
 {'id': 'HeaderV2.size', 'value': 0}]

In [25]:
header.to_json()

'{"type": {"version": 0, "id": 0, "length": 0, "module": "\\u0000"}, "size": 0}'

In [26]:
to_yaml(header)

type:
  version: 0
  id: 0
  length: 0
  module: "\0"
size: 0


## Create a Structure on the Fly

In [27]:
structure = Structure()
structure.version = Decimal(8, align_to=4)
structure.id = Decimal(8, align_to=4)
structure.length = Decimal(8, align_to=4)
structure.module = Char(align_to=4)

In [28]:
structure.to_list()

[('Structure.version', 0),
 ('Structure.id', 0),
 ('Structure.length', 0),
 ('Structure.module', '\x00')]

In [29]:
structure.to_csv()

[{'id': 'Structure.version', 'value': 0},
 {'id': 'Structure.id', 'value': 0},
 {'id': 'Structure.length', 'value': 0},
 {'id': 'Structure.module', 'value': '\x00'}]

In [30]:
structure.to_json()

'{"version": 0, "id": 0, "length": 0, "module": "\\u0000"}'

In [31]:
to_yaml(structure, flow_style=False)

version: 0
id: 0
length: 0
module: "\0"


## Create a Structure with Keywords

In [32]:
structure = Structure(
    version=Decimal(8, 4),
    id=Decimal(8, 4), 
    length=Decimal(8, 4), 
    module=Char(4))

In [33]:
structure.to_list()

[('Structure.version', 0),
 ('Structure.id', 0),
 ('Structure.length', 0),
 ('Structure.module', '\x00')]

In [34]:
structure.to_csv()

[{'id': 'Structure.version', 'value': 0},
 {'id': 'Structure.id', 'value': 0},
 {'id': 'Structure.length', 'value': 0},
 {'id': 'Structure.module', 'value': '\x00'}]

In [35]:
structure.to_json()

'{"version": 0, "id": 0, "length": 0, "module": "\\u0000"}'

In [36]:
to_yaml(structure, flow_style=False)

version: 0
id: 0
length: 0
module: "\0"


## Nest Structures

In [37]:
structure = Structure(
    type=Structure(version=Decimal(8, 4), 
                   id=Decimal(8, 4), 
                   length=Decimal(8, 4), 
                   module=Char(4)),
    size=Decimal32())

In [38]:
structure.to_list()

[('Structure.type.version', 0),
 ('Structure.type.id', 0),
 ('Structure.type.length', 0),
 ('Structure.type.module', '\x00'),
 ('Structure.size', 0)]

In [39]:
structure.to_csv()

[{'id': 'Structure.type.version', 'value': 0},
 {'id': 'Structure.type.id', 'value': 0},
 {'id': 'Structure.type.length', 'value': 0},
 {'id': 'Structure.type.module', 'value': '\x00'},
 {'id': 'Structure.size', 'value': 0}]

In [40]:
structure.to_json()

'{"type": {"version": 0, "id": 0, "length": 0, "module": "\\u0000"}, "size": 0}'

In [41]:
to_yaml(structure, flow_style=False)

type:
  version: 0
  id: 0
  length: 0
  module: "\0"
size: 0


## Initialize a Structure

In [42]:
structure = Structure(
    version=Decimal(8, 4), 
    id=Decimal(8, 4), 
    length=Decimal(8, 4), 
    module=Char(4))

In [43]:
structure.to_list()

[('Structure.version', 0),
 ('Structure.id', 0),
 ('Structure.length', 0),
 ('Structure.module', '\x00')]

In [44]:
structure.to_csv()

[{'id': 'Structure.version', 'value': 0},
 {'id': 'Structure.id', 'value': 0},
 {'id': 'Structure.length', 'value': 0},
 {'id': 'Structure.module', 'value': '\x00'}]

In [45]:
structure.to_json()

'{"version": 0, "id": 0, "length": 0, "module": "\\u0000"}'

In [46]:
to_yaml(structure, flow_style=False)

version: 0
id: 0
length: 0
module: "\0"


In [47]:
structure.initialize_fields(dict(version=1, id=2, length=9, module='F'))

In [48]:
structure.initialize_fields({"version": 1, "id": 2, "length": 9, "module": "F"})

In [49]:
structure.to_list()

[('Structure.version', 1),
 ('Structure.id', 2),
 ('Structure.length', 9),
 ('Structure.module', 'F')]

In [50]:
structure.to_csv()

[{'id': 'Structure.version', 'value': 1},
 {'id': 'Structure.id', 'value': 2},
 {'id': 'Structure.length', 'value': 9},
 {'id': 'Structure.module', 'value': 'F'}]

In [51]:
structure.to_json()

'{"version": 1, "id": 2, "length": 9, "module": "F"}'

In [52]:
to_yaml(structure, flow_style=False)

version: 1
id: 2
length: 9
module: F


## Display a Structure

In [53]:
structure

{'version': Decimal(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=8, value=1),
 'id': Decimal(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=8, value=2),
 'length': Decimal(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=8, value=9),
 'module': Char(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=8, value='F')}

## Metadata of a Structure

In [54]:
structure.describe()

{'class': 'Structure',
 'name': 'Structure',
 'size': 4,
 'type': 'Structure',
 'member': [{'address': 0,
   'alignment': [4, 0],
   'class': 'Decimal8',
   'index': [0, 0],
   'max': 255,
   'min': 0,
   'name': 'version',
   'order': 'auto',
   'signed': False,
   'size': 8,
   'type': 'Field',
   'value': 1},
  {'address': 0,
   'alignment': [4, 0],
   'class': 'Decimal8',
   'index': [0, 0],
   'max': 255,
   'min': 0,
   'name': 'id',
   'order': 'auto',
   'signed': False,
   'size': 8,
   'type': 'Field',
   'value': 2},
  {'address': 0,
   'alignment': [4, 0],
   'class': 'Decimal8',
   'index': [0, 0],
   'max': 255,
   'min': 0,
   'name': 'length',
   'order': 'auto',
   'signed': False,
   'size': 8,
   'type': 'Field',
   'value': 9},
  {'address': 0,
   'alignment': [4, 0],
   'class': 'Char',
   'index': [0, 0],
   'max': 255,
   'min': 0,
   'name': 'module',
   'order': 'auto',
   'signed': False,
   'size': 8,
   'type': 'Field',
   'value': 'F'}]}

In [55]:
json.dump(structure.describe(), sys.stdout, indent=2)

{
  "class": "Structure",
  "name": "Structure",
  "size": 4,
  "type": "Structure",
  "member": [
    {
      "address": 0,
      "alignment": [
        4,
        0
      ],
      "class": "Decimal8",
      "index": [
        0,
        0
      ],
      "max": 255,
      "min": 0,
      "name": "version",
      "order": "auto",
      "signed": false,
      "size": 8,
      "type": "Field",
      "value": 1
    },
    {
      "address": 0,
      "alignment": [
        4,
        0
      ],
      "class": "Decimal8",
      "index": [
        0,
        0
      ],
      "max": 255,
      "min": 0,
      "name": "id",
      "order": "auto",
      "signed": false,
      "size": 8,
      "type": "Field",
      "value": 2
    },
    {
      "address": 0,
      "alignment": [
        4,
        0
      ],
      "class": "Decimal8",
      "index": [
        0,
        0
      ],
      "max": 255,
      "min": 0,
      "name": "length",
      "order": "auto",
      "signed": false,
      "size

In [56]:
d3flare_json(structure.describe(), sys.stdout, indent=2)

{
  "class": "Structure",
  "name": "Structure",
  "children": [
    {
      "class": "Decimal8",
      "name": "version",
      "size": 8,
      "value": 1
    },
    {
      "class": "Decimal8",
      "name": "id",
      "size": 8,
      "value": 2
    },
    {
      "class": "Decimal8",
      "name": "length",
      "size": 8,
      "value": 9
    },
    {
      "class": "Char",
      "name": "module",
      "size": 8,
      "value": "F"
    }
  ]
}

## Size of a Strucutrue

In [57]:
structure.container_size()

(4, 0)

In [58]:
num_of_bytes, num_of_remaining_bits = structure.container_size()

In [59]:
num_of_bytes

4

In [60]:
num_of_remaining_bits

0

## Indexing

In [61]:
structure.to_list('index')

[('Structure.version',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Structure.id',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Structure.length',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Structure.module',
  Index(byte=0, bit=0, address=0, base_address=0, update=False))]

In [62]:
structure.to_csv('index.byte', 'index.bit', 'index.address', fieldnames=('id', 'index', 'offset', 'address'))

[{'id': 'Structure.version', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Structure.id', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Structure.length', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Structure.module', 'index': 0, 'offset': 0, 'address': 0}]

In [63]:
structure.to_json('index')

'{"version": [0, 0, 0, 0, false], "id": [0, 0, 0, 0, false], "length": [0, 0, 0, 0, false], "module": [0, 0, 0, 0, false]}'

In [64]:
structure.index_fields(index=Index())

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [65]:
structure.index_fields()

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [66]:
structure.to_list('index')

[('Structure.version',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Structure.id',
  Index(byte=0, bit=8, address=0, base_address=0, update=False)),
 ('Structure.length',
  Index(byte=0, bit=16, address=0, base_address=0, update=False)),
 ('Structure.module',
  Index(byte=0, bit=24, address=0, base_address=0, update=False))]

In [67]:
structure.to_csv('index.byte', 'index.bit', 'index.address', fieldnames=('id', 'index', 'offset', 'address'))

[{'id': 'Structure.version', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Structure.id', 'index': 0, 'offset': 8, 'address': 0},
 {'id': 'Structure.length', 'index': 0, 'offset': 16, 'address': 0},
 {'id': 'Structure.module', 'index': 0, 'offset': 24, 'address': 0}]

In [68]:
structure.to_json('index')

'{"version": [0, 0, 0, 0, false], "id": [0, 8, 0, 0, false], "length": [0, 16, 0, 0, false], "module": [0, 24, 0, 0, false]}'

## De-Serializing

In [69]:
structure.deserialize(bytes.fromhex('01020946f00f00'))

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [70]:
structure.to_list()

[('Structure.version', 1),
 ('Structure.id', 2),
 ('Structure.length', 9),
 ('Structure.module', 'F')]

In [71]:
structure.to_csv()

[{'id': 'Structure.version', 'value': 1},
 {'id': 'Structure.id', 'value': 2},
 {'id': 'Structure.length', 'value': 9},
 {'id': 'Structure.module', 'value': 'F'}]

In [72]:
structure.to_json()

'{"version": 1, "id": 2, "length": 9, "module": "F"}'

In [73]:
to_yaml(structure, flow_style=False)

version: 1
id: 2
length: 9
module: F


## Serializing

In [74]:
bytestream = bytearray()
bytestream.hex()

''

In [75]:
structure.serialize(bytestream)

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [76]:
bytes(structure).hex()

'01020946'

In [77]:
bytestream.hex()

'01020946'

## Structure Members

### Number of Members

In [78]:
len(structure)

4

### Access a Member

In [79]:
structure.version

Decimal(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=8, value=1)

In [80]:
structure['version']

Decimal(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=8, value=1)

### Access the Attributes of a Member Field

In [81]:
structure.version.name

'Decimal8'

In [82]:
structure.version.value

1

In [83]:
structure.version.bit_size

8

In [84]:
structure.version.alignment

Alignment(byte_size=4, bit_offset=0)

In [85]:
structure.version.alignment.byte_size

4

In [86]:
structure.version.alignment.bit_offset

0

In [87]:
structure.version.byte_order

Byteorder.auto = 'auto'

In [88]:
structure.version.byte_order.value

'auto'

In [89]:
structure.version.index

Index(byte=0, bit=0, address=0, base_address=0, update=False)

In [90]:
structure.version.index.byte

0

In [91]:
structure.version.index.bit

0

In [92]:
structure.version.index.address

0

In [93]:
structure.version.index.base_address

0

In [94]:
structure.version.index.update

False

### List the Member Items

In [95]:
[(name, member) for name, member in structure.items()]

[('version',
  Decimal(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=8, value=1)),
 ('id',
  Decimal(index=Index(byte=0, bit=8, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=8), bit_size=8, value=2)),
 ('length',
  Decimal(index=Index(byte=0, bit=16, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=16), bit_size=8, value=9)),
 ('module',
  Char(index=Index(byte=0, bit=24, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=24), bit_size=8, value='F'))]

### List the Member Names

In [96]:
[name for name in structure.keys()]

['version', 'id', 'length', 'module']

### List the Members

In [97]:
[member for member in structure.values()]

[Decimal(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=8, value=1),
 Decimal(index=Index(byte=0, bit=8, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=8), bit_size=8, value=2),
 Decimal(index=Index(byte=0, bit=16, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=16), bit_size=8, value=9),
 Char(index=Index(byte=0, bit=24, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=24), bit_size=8, value='F')]

## Structure Fields

### View Field Attributes

In [98]:
structure.view_fields()

{'version': 1, 'id': 2, 'length': 9, 'module': 'F'}

In [99]:
structure.view_fields('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order.name')

{'version': {'name': 'Decimal8',
  'bit_size': 8,
  'value': 1,
  'index': Index(byte=0, bit=0, address=0, base_address=0, update=False),
  'alignment': Alignment(byte_size=4, bit_offset=0),
  'byte_order.name': 'auto'},
 'id': {'name': 'Decimal8',
  'bit_size': 8,
  'value': 2,
  'index': Index(byte=0, bit=8, address=0, base_address=0, update=False),
  'alignment': Alignment(byte_size=4, bit_offset=8),
  'byte_order.name': 'auto'},
 'length': {'name': 'Decimal8',
  'bit_size': 8,
  'value': 9,
  'index': Index(byte=0, bit=16, address=0, base_address=0, update=False),
  'alignment': Alignment(byte_size=4, bit_offset=16),
  'byte_order.name': 'auto'},
 'module': {'name': 'Char',
  'bit_size': 8,
  'value': 'F',
  'index': Index(byte=0, bit=24, address=0, base_address=0, update=False),
  'alignment': Alignment(byte_size=4, bit_offset=24),
  'byte_order.name': 'auto'}}

### View as a JSON string

In [100]:
structure.to_json()

'{"version": 1, "id": 2, "length": 9, "module": "F"}'

In [101]:
print(structure.to_json(indent=2))

{
  "version": 1,
  "id": 2,
  "length": 9,
  "module": "F"
}


In [102]:
structure.to_json('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order')

'{"version": {"name": "Decimal8", "bit_size": 8, "value": 1, "index": [0, 0, 0, 0, false], "alignment": [4, 0], "byte_order": "auto"}, "id": {"name": "Decimal8", "bit_size": 8, "value": 2, "index": [0, 8, 0, 0, false], "alignment": [4, 8], "byte_order": "auto"}, "length": {"name": "Decimal8", "bit_size": 8, "value": 9, "index": [0, 16, 0, 0, false], "alignment": [4, 16], "byte_order": "auto"}, "module": {"name": "Char", "bit_size": 8, "value": "F", "index": [0, 24, 0, 0, false], "alignment": [4, 24], "byte_order": "auto"}}'

In [103]:
print(structure.to_json('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order', indent=2))

{
  "version": {
    "name": "Decimal8",
    "bit_size": 8,
    "value": 1,
    "index": [
      0,
      0,
      0,
      0,
      false
    ],
    "alignment": [
      4,
      0
    ],
    "byte_order": "auto"
  },
  "id": {
    "name": "Decimal8",
    "bit_size": 8,
    "value": 2,
    "index": [
      0,
      8,
      0,
      0,
      false
    ],
    "alignment": [
      4,
      8
    ],
    "byte_order": "auto"
  },
  "length": {
    "name": "Decimal8",
    "bit_size": 8,
    "value": 9,
    "index": [
      0,
      16,
      0,
      0,
      false
    ],
    "alignment": [
      4,
      16
    ],
    "byte_order": "auto"
  },
  "module": {
    "name": "Char",
    "bit_size": 8,
    "value": "F",
    "index": [
      0,
      24,
      0,
      0,
      false
    ],
    "alignment": [
      4,
      24
    ],
    "byte_order": "auto"
  }
}


### Write to a JSON file

In [104]:
structure.write_json('Structure.json')

### List the Field Items

In [105]:
structure.field_items()

[('version',
  Decimal(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=8, value=1)),
 ('id',
  Decimal(index=Index(byte=0, bit=8, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=8), bit_size=8, value=2)),
 ('length',
  Decimal(index=Index(byte=0, bit=16, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=16), bit_size=8, value=9)),
 ('module',
  Char(index=Index(byte=0, bit=24, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=24), bit_size=8, value='F'))]

### List the Field Attributes

In [106]:
structure.to_list()

[('Structure.version', 1),
 ('Structure.id', 2),
 ('Structure.length', 9),
 ('Structure.module', 'F')]

In [107]:
structure.to_list('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order.name')

[('Structure.version',
  ('Decimal8',
   8,
   1,
   Index(byte=0, bit=0, address=0, base_address=0, update=False),
   Alignment(byte_size=4, bit_offset=0),
   'auto')),
 ('Structure.id',
  ('Decimal8',
   8,
   2,
   Index(byte=0, bit=8, address=0, base_address=0, update=False),
   Alignment(byte_size=4, bit_offset=8),
   'auto')),
 ('Structure.length',
  ('Decimal8',
   8,
   9,
   Index(byte=0, bit=16, address=0, base_address=0, update=False),
   Alignment(byte_size=4, bit_offset=16),
   'auto')),
 ('Structure.module',
  ('Char',
   8,
   'F',
   Index(byte=0, bit=24, address=0, base_address=0, update=False),
   Alignment(byte_size=4, bit_offset=24),
   'auto'))]

In [108]:
structure.to_dict()

{'Structure': {'version': 1, 'id': 2, 'length': 9, 'module': 'F'}}

In [109]:
structure.to_dict('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order')

{'Structure': {'version': ('Decimal8',
   8,
   1,
   Index(byte=0, bit=0, address=0, base_address=0, update=False),
   Alignment(byte_size=4, bit_offset=0),
   Byteorder.auto = 'auto'),
  'id': ('Decimal8',
   8,
   2,
   Index(byte=0, bit=8, address=0, base_address=0, update=False),
   Alignment(byte_size=4, bit_offset=8),
   Byteorder.auto = 'auto'),
  'length': ('Decimal8',
   8,
   9,
   Index(byte=0, bit=16, address=0, base_address=0, update=False),
   Alignment(byte_size=4, bit_offset=16),
   Byteorder.auto = 'auto'),
  'module': ('Char',
   8,
   'F',
   Index(byte=0, bit=24, address=0, base_address=0, update=False),
   Alignment(byte_size=4, bit_offset=24),
   Byteorder.auto = 'auto')}}

In [110]:
structure.to_csv()

[{'id': 'Structure.version', 'value': 1},
 {'id': 'Structure.id', 'value': 2},
 {'id': 'Structure.length', 'value': 9},
 {'id': 'Structure.module', 'value': 'F'}]

In [111]:
structure.to_csv('name', 'bit_size', 'value', fieldnames=('id', 'type', 'size', 'value'))

[{'id': 'Structure.version', 'type': 'Decimal8', 'size': 8, 'value': 1},
 {'id': 'Structure.id', 'type': 'Decimal8', 'size': 8, 'value': 2},
 {'id': 'Structure.length', 'type': 'Decimal8', 'size': 8, 'value': 9},
 {'id': 'Structure.module', 'type': 'Char', 'size': 8, 'value': 'F'}]

### Write the Field Attributes to a `.csv` File

In [112]:
structure.write_csv('Structure.csv', 'name', 'bit_size', 'value', fieldnames=('id', 'type', 'size', 'value'))

### Save the Field Attributes to an `.ini` File

In [113]:
structure.save('Structure.ini')

### Load the Field Value from an `.ini` File

In [114]:
structure.load('Structure.ini')

[Structure]
Structure.version = 1
Structure.id = 2
Structure.length = 9
Structure.module = F
